In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
import os

from taogpt.orchestrator import *
from taogpt.utils import *
from taogpt.llm_model import LangChainLLM
from taogpt.prompts import PromptDb
import taogpt.utils as utils

utils.enable_debugging(0)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
TEMPERATURE = 0.7

In [4]:
with open(os.path.join(os.environ['HOME'], '.ssh', 'openai-zillow.json'), 'r') as f:
    credentials = json.load(f)
os.environ["OPENAI_API_KEY"] = credentials['key']
os.environ["OPENAI_API_BASE"] = credentials['url']
llm3_5 = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=TEMPERATURE)
llm4 = ChatOpenAI(model_name='gpt-4-32k', temperature=TEMPERATURE)

conversation = ConversationChain(llm=llm3_5)
conversation.predict(input="What's your model version?")

"I am using OpenAI's GPT-3 model, specifically version 3.0.6."

In [5]:
llm3_5.model_name, llm4.model_name

('gpt-3.5-turbo-16k', 'gpt-4-32k')

In [24]:
prompts = PromptDb.load_defaults()
logger = MarkdownLogger('logs/taogpt_log.md')
config = Config(
    ask_user_before_execute_codes=False,
    ask_user_questions_in_one_prompt=True,
    initial_expansion=2,
    max_tree_branches=8,
    max_tokens=20000,
    check_final=True
)
executor = Orchestrator(
    config=config,
    llm=LangChainLLM(llm4, logger=logger),
    prompts=prompts,
    markdown_logger=logger,
    sage_llm=LangChainLLM(llm4, logger=logger),
)

experiment_name = 'example'
executor.start("""
Given a game board of Weiqi, create a Python program to calculate the scores of each player. Use the Chinese rules
for scoring.
""", analyze_first=True)
# executor.start("""
# Solve this Game of 24: 7, 6, 2, 1
# (Use each number exactly once. Allowed operations: +, -, *, /, and parentheses.)
# """, analyze_first=True)


KeyboardInterrupt: User cancelled

In [23]:
executor.resume(10000)

extend token allowance by 10000 to 40000


RuntimeError: Regular LLM consumed 40059 tokens, exceeded allowance of 40000

In [9]:
logger = MarkdownLogger(f'examples/{experiment_name}.final.md')
logger.log_conversation(executor.show_conversation_thread(with_extras=True))
logger.log(f"**total tokens**: {executor.llm.total_tokens}")